In [1]:
%pip install -r ../requirements.txt
path = '../path.txt'
with open(path, 'r') as archivo:
    for linea in archivo.readlines():
        exec(linea.strip(), globals())


  Using cached asttokens-2.4.1-py2.py3-none-any.whl (27 kB)
  Using cached branca-0.7.2-py3-none-any.whl (25 kB)
  Using cached comm-0.2.2-py3-none-any.whl (7.2 kB)
  Using cached contourpy-1.2.1-cp311-cp311-win_amd64.whl (188 kB)
  Using cached debugpy-1.8.1-cp311-cp311-win_amd64.whl (4.7 MB)
  Using cached executing-2.0.1-py2.py3-none-any.whl (24 kB)
  Using cached fonttools-4.51.0-cp311-cp311-win_amd64.whl (2.2 MB)
  Using cached ipykernel-6.29.4-py3-none-any.whl (117 kB)
  Using cached ipython-8.24.0-py3-none-any.whl (816 kB)
  Using cached jedi-0.19.1-py2.py3-none-any.whl (1.6 MB)
  Using cached jinja2-3.1.4-py3-none-any.whl (133 kB)
  Using cached jupyter_client-8.6.1-py3-none-any.whl (105 kB)
  Using cached jupyter_core-5.7.2-py3-none-any.whl (28 kB)
  Using cached matplotlib-3.8.4-cp311-cp311-win_amd64.whl (7.7 MB)
  Using cached matplotlib_inline-0.1.7-py3-none-any.whl (9.9 kB)
  Using cached nest_asyncio-1.6.0-py3-none-any.whl (5.2 kB)
  Using cached pandas-2.0.3-cp311-cp311

ERROR: Could not install packages due to an OSError: [WinError 5] Acceso denegado: 'C:\\Users\\ANGEL\\AppData\\Local\\Programs\\Python\\Python311\\Lib\\site-packages\\~yarrow\\arrow.dll'
Consider using the `--user` option or check the permissions.


[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


## Árbol de regresión

EL presente modelo busca predecir el precio que tiene un servicio de taxi tomando en cuenta parametros como: distancia del viaje y tiempo del recorrido.

Se importan las librerias necesarias

In [2]:
import pandas as pd
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from joblib import dump


Se hace el entrenamiento con 3 datasets de recorridos de taxis ecologicos para tres meses consecutivos del año 2023, se concatenan y se usan para el entrenamiento

In [3]:
d0 = pd.read_parquet(df0)
df1 = pd.read_parquet(df1)
df2 = pd.read_parquet(df2)
df = pd.concat([d0, df1, df2])

In [4]:
df.shape

(183020, 12)

Se filtra el data set con los valores de la columna 'payment type' mayor que -1 ya que -1 representa a los valores nulos

In [5]:
df = df[df['payment_type'] > -1]
df.shape

(115890, 12)

In [6]:
df.head(1)

,start_trip,end_trip,pu_location_id,do_location_id,passenger_count,trip_distance,fare_amount,tip_amount,total_amount,payment_type,congestion_surcharge,type_of_taxi
0,2023-10-01 00:57:33,2023-10-01 01:07:58,166,74,1,1.45,12.1,2.92,17.52,1,0,1


In [7]:
df.sample(5)

,start_trip,end_trip,pu_location_id,do_location_id,passenger_count,trip_distance,fare_amount,tip_amount,total_amount,payment_type,congestion_surcharge,type_of_taxi
25796,2023-11-13 17:45:33,2023-11-13 17:51:55,210,210,1,0.70,6.5,2.10,12.60,1,0,1
45098,2023-10-23 20:16:18,2023-10-23 20:26:39,134,216,1,2.28,12.8,3.82,19.12,1,0,1
14867,2023-11-08 12:43:10,2023-11-08 13:05:39,24,143,2,2.44,19.8,6.01,30.06,1,2,1
26975,2023-10-14 18:01:32,2023-10-14 18:32:11,25,195,1,1.99,24.0,4.50,30.00,1,0,1
4575,2023-11-02 21:27:29,2023-11-02 21:40:55,130,205,1,2.92,17.0,0.00,18.00,1,0,1


Se extrate informacion de las fechas de inicio y fin de viaje para generar columnas con los siguientes valores: año, mes, dia y minutos de viaje

In [8]:
df['year'] = df['start_trip'].dt.year
df['month'] = df['start_trip'].dt.month
df['day'] = df['start_trip'].dt.dayofweek
df['min_trip'] = (df['end_trip'] - df['start_trip']).dt.total_seconds() / 60
df.head(1)

,start_trip,end_trip,pu_location_id,do_location_id,passenger_count,trip_distance,fare_amount,tip_amount,total_amount,payment_type,congestion_surcharge,type_of_taxi,year,month,day,min_trip
0,2023-10-01 00:57:33,2023-10-01 01:07:58,166,74,1,1.45,12.1,2.92,17.52,1,0,1,2023,10,6,10.416667


In [9]:
# Arbol de decision

In [10]:
# Selección de las variables independientes
X = df[['min_trip', 'trip_distance']]


# Selección de la variable dependiente
y = df['fare_amount']

# Separación del conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Creación del clasificador de árbol de decisión
arbol = tree.DecisionTreeRegressor(max_depth = 8, random_state=42, )

# Entrenamiento del modelo
arbol.fit(X_train, y_train)

# Evaluación del modelo
score = arbol.score(X_test, y_test)

# Predicción del recargo por congestión
predicciones = arbol.predict(X_test)


In [11]:
# Cálculo del error cuadrático medio (MSE)
mse = mean_squared_error(y_test, predicciones)

# Cálculo del R cuadrado (R^2)
r2 = r2_score(y_test, predicciones)

# Impresión de las métricas de evaluación
print("MSE:", mse)
print("R^2:", r2)

MSE: 19.317563136578976
R^2: 0.8556884306103201


In [12]:
resultados = pd.DataFrame({'Valor real': y_test, 'Valor predicho': predicciones}).head(5)
print(score)

# Impresión del dataframe
print(resultados.to_string())

0.8556884306103201
       Valor real  Valor predicho
28213         5.1        6.119324
44017         8.6        9.668947
53115        10.0       10.308712
6444         21.2       21.272973
45306         7.2        7.202893


Se observa un Score de 0.85, y se visualiza una columna con valores reales y valores predichos

Se guarda el modelo en formato joblib, para ser posteriormente consumido por Streamlit

In [13]:
dump(arbol, modelo_precio_entrenado)

['../modelos_ML/modelos_entrenados/modelo_precio_entrenado.joblib']